# 🏁 Initialize

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import datetime
from pprint import pprint

PATH = os.path.abspath('')

## Twitter tweepy client

In [2]:
API_KEY = os.getenv('TWITTER_BOT_KEY')
API_SECRET_KEY = os.getenv('TWITTER_BOT_SECRET_KEY')

ACCESS_TOKEN = os.getenv('rbgtx_ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.getenv('rbgtx_ACCESS_TOKEN_SECRET')

In [3]:
import tweepy

auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
try:
    api.verify_credentials()
except:
    raise Exception("Authentication not OK")

me = api.me()
me.screen_name

'ReebotGTX'

## MongoDB client

In [4]:
MONG_USER = os.getenv('MONG_USER')
MONG_PWD  = os.getenv('MONG_PWD')
MONG_HOST = os.getenv('MONG_HOST')
MONG_PORT = os.getenv('MONG_PORT')
DATABASE = "dataDumpingGround"

In [5]:
from pymongo import MongoClient
from bson.objectid import ObjectId

uri = f"mongodb://{MONG_USER}:{MONG_PWD}@{MONG_HOST}:{MONG_PORT}/{DATABASE}"
client = MongoClient(uri)
db = client[DATABASE]
db

Database(MongoClient(host=['171.244.50.232:27017'], document_class=dict, tz_aware=False, connect=True), 'dataDumpingGround')

# 🎬 Action

In [6]:
import pprint
from helperfunctions import (
    compress_object,
    prettyprint,
    strip_tags
)
from datetime import datetime, timezone, timedelta

## 📐 Params

In [7]:
from sys import maxsize as inf
LIMIT = 1#inf
USER_ID = 'reebotgtx'

## Get user info

In [8]:
user = api.get_user(USER_ID)
user_detail = user._json

In [9]:
# https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/user
attrs_to_save = [
    'id', 'name', 'screen_name', 'location', 'url', 'description',
    'protected', 'verified', 'followers_count', 'friends_count', 'listed_count',
    'favourites_count', 'statuses_count', 'created_at', 'profile_banner_url', 
    'profile_image_url_https', 'default_profile', 'default_profile_image',
    'withheld_in_countries', 'withheld_scope'
]

my_user_obj = compress_object(user._json, attrs_to_save)
my_user_obj['Last updated'] = datetime.now(tz=timezone(timedelta(hours=+7)))
my_user_obj

{'Absent keys': ['withheld_scope'],
 'id': 1373551635499085827,
 'name': 'Reebot GTX',
 'screen_name': 'ReebotGTX',
 'location': '',
 'url': 'None',
 'description': 'personal multi purpose bot account by @lreeeon',
 'protected': 'False',
 'verified': 'False',
 'followers_count': 2,
 'friends_count': 1,
 'listed_count': 0,
 'favourites_count': 1,
 'statuses_count': 32,
 'created_at': 'Sun Mar 21 08:26:54 +0000 2021',
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/1373551635499085827/1616316092',
 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1373553692754862082/5kA93Njm_normal.png',
 'default_profile': 'True',
 'default_profile_image': 'False',
 'withheld_in_countries': '[]',
 'Last updated': datetime.datetime(2021, 6, 18, 17, 20, 32, 453756, tzinfo=datetime.timezone(datetime.timedelta(seconds=25200)))}

Save to collection "profile"

In [10]:
profile_col = db["twitterProfiles"]
key = {'id': my_user_obj['id']}
profile_col.update(key, my_user_obj, upsert=True)

<ipython-input-10-28205dcc945f>:3: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  profile_col.update(key, my_user_obj, upsert=True)


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

## Get tweets

In [11]:
'''
with open(os.path.join(PATH, f'{USER_ID}_tweet_dump.txt'), 'w+', encoding='utf8') as file:
    for tweet in tweepy.Cursor(api.user_timeline, id=USER_ID).items(LIMIT):
        # parsed = json.loads(tweet._json)
        print(json.dumps(tweet._json, indent=4, sort_keys=True))
        # print(tweet)
        # print(dir(tweet))
        file.write(tweet.text +'\n')
'''
pass

In [12]:
# https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet
attrs = [
    'id', 'text', 'attachments', 'author_id', 'conversation_id', 'created_at',
    'geo', 'in_reply_to_user_id', 'lang', 'possiby_sensitive', 'reply_settings', 
    'source', 'withheld',
    'entities', 'non_public_metrics', 'organic_metrics', 'promoted_metrics', 'public_metrics',
    'referenced_tweets', 'context_annotations', 
]
# https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet
attrs = [
    'created_at', 'id', 'full_text', 'coordinates', 'place',
    'in_reply_to_status_id', 'in_reply_to_user_id', 'quoted_status_id', 'quoted_status',
    'quote_count', 'reply_count', 'retweet_count', 'favorite_count',
    'entities', 'extended_entities', 'possibly_sensitive', 'filter_level', 'lang', 'matching_rules'
]

In [13]:
from tqdm.notebook import tqdm

tweet_col = db["twitterTweets"]

for tweet in tqdm( tweepy.Cursor(api.user_timeline, id=USER_ID, tweet_mode='extended').items(LIMIT) ):
    tweet_obj = compress_object(tweet._json, attrs)
    tweet_obj['Last updated'] = datetime.now(tz=timezone(timedelta(hours=+7)))
    tweet_obj['Author_id'] = tweet._json['user']['id']
    tweet_obj['Source'] = strip_tags(tweet._json['source'])
    tweet_obj['Retweeted_status_id'] = tweet._json['retweeted_status']['id'] if 'retweeted_status' in tweet._json else None

    # prettyprint(tweet._json)
    # prettyprint(tweet_obj)

    key = {'id': tweet_obj['id']}
    tweet_col.update(key, tweet_obj, upsert=True)
"Ok"

0it [00:00, ?it/s]

<ipython-input-13-eabd64ab62b4>:16: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  tweet_col.update(key, tweet_obj, upsert=True)


'Ok'

In [14]:
pprint(tweet._json )

TypeError: 'module' object is not callable